In [27]:
from ordinal_classifier import OrdinalClassifier
import pandas as pd
from DashApp.func import Classifier
import plotly.graph_objects as go
import plotly.express as px
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import sklearn.metrics
from sklearn import svm
import copy
from sklearn.model_selection import cross_val_predict, cross_validate
df = pd.read_pickle(r'data\data_df.pkl')
df.head()

,OriginalText,CleanedText,WordList,Sentyment,Lubienie,Wstręt,Złość,Strach,Zaskoczenie,Oczekiwanie,Radość,Smutek,RemovedSW,Sentyment_nom
AF1.txt,Wczoraj rano przytrafiło mi się coś miłego. Pr...,wczoraj rano przytrafiło mi się coś miłego prz...,"[wczoraj, rano, przytrafiło, mi, się, coś, mił...",5.0,2.0,0.0,0.0,0.0,3.0,0.0,2.0,0.0,wczoraj rano przytrafiło miłego przechodziłam ...,1
AF10.txt,"Pan Dussel, człowiek, o którym zawsze mówiono,...",pan dussel człowiek o którym zawsze mówiono że...,"[pan, dussel, człowiek, o, którym, zawsze, mów...",1.0,0.0,1.0,4.0,0.0,0.0,1.0,0.0,2.0,dussel człowiek mówiono umie świetnie radzić d...,-1
AF11.txt,Jakbym w ciągu dnia nie słyszała wystarczająco...,jakbym w ciągu dnia nie słyszała wystarczająco...,"[jakbym, w, ciągu, dnia, nie, słyszała, wystar...",1.0,0.0,1.0,3.0,1.0,0.0,1.0,0.0,2.0,jakbym ciągu dnia słyszała wystarczająco „psst...,-1
AF12.txt,"Pienię się z wściekłości, a nie wolno mi tego ...",pienię się z wściekłości a nie wolno mi tego o...,"[pienię, się, z, wściekłości, a, nie, wolno, m...",1.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0,pienię wściekłości wolno okazać chciałabym tup...,-1
AF13.txt,"Wszyscy uważają, że jestem przesadna, gdy mówi...",wszyscy uważają że jestem przesadna gdy mówię ...,"[wszyscy, uważają, że, jestem, przesadna, gdy,...",1.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,3.0,uważają przesadna mówię śmieszna milczę bezcze...,-1


In [28]:
emotions = ['Lubienie', 'Wstręt', 'Złość', 'Strach', 'Zaskoczenie', 'Oczekiwanie', 'Radość',
                         'Smutek']
X = df['RemovedSW']
vector = CountVectorizer()
X_vec = vector.fit_transform(X).toarray()
def predict_values_vec(X_vec, emotions, clf, df):
    val_dict = {}
    for emo in emotions:
        ylabels = df[emo]
        X_train, X_test, y_train, y_test = train_test_split(X_vec, ylabels, test_size=0.3)
        ord_clf = OrdinalClassifier(clf)
        ord_clf.fit(X_train, y_train)
        y_pred = ord_clf.predict(X_test)
        val_dict[emo] = (y_pred, y_test)

    return val_dict



def calc_acc(X_vec, emotions, clf, df):
    dict = {}
    val_dict = predict_values_vec(X_vec, emotions, clf, df)
    for k in val_dict.keys():
        y_pred, y_true = val_dict[k]
        dict[k] = {'test_accuracy': accuracy_score(y_true, y_pred), 'test_f1_macro' : f1_score(y_true, y_pred, average='macro'), 'test_recall_macro' : recall_score(y_true, y_pred, average='macro'), 'test_precision_macro' : precision_score(y_true, y_pred, average='macro', zero_division=0)}
    return dict

In [29]:
score_df = pd.DataFrame(calc_acc(X_vec, emotions, MultinomialNB(), df))
score_df.head()

,Lubienie,Wstręt,Złość,Strach,Zaskoczenie,Oczekiwanie,Radość,Smutek
test_accuracy,0.225806,0.258065,0.225806,0.258065,0.161290,0.193548,0.483871,0.258065
test_f1_macro,0.134459,0.244398,0.146667,0.201255,0.128024,0.107792,0.183590,0.242892
test_recall_macro,0.296923,0.347143,0.260000,0.303333,0.195000,0.258824,0.333333,0.279394
test_precision_macro,0.280952,0.328571,0.160000,0.222857,0.124848,0.110909,0.177778,0.234444


In [30]:

score_df.rename(index={'test_accuracy': 'accuracy', 'test_f1_macro' : 'f1', 'test_recall_macro' : 'recall', 'test_precision_macro' : 'precision'}, inplace=True)
score_df = score_df.round(decimals=2)
score_df.head()

,Lubienie,Wstręt,Złość,Strach,Zaskoczenie,Oczekiwanie,Radość,Smutek
accuracy,0.23,0.26,0.23,0.26,0.16,0.19,0.48,0.26
f1,0.13,0.24,0.15,0.20,0.13,0.11,0.18,0.24
recall,0.30,0.35,0.26,0.30,0.20,0.26,0.33,0.28
precision,0.28,0.33,0.16,0.22,0.12,0.11,0.18,0.23


In [31]:
score_df.to_pickle(r'data\MNB_noSW_ord_score.pkl')